<a href="https://colab.research.google.com/github/HuangJin-De/Machine-Learning-in-Atmospheric-Thermodynamics/blob/master/hw03/hw03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! rm -rf Machine-Learning-in-Atmospheric-Thermodynamics
! git clone https://github.com/HuangJin-De/Machine-Learning-in-Atmospheric-Thermodynamics.git



Cloning into 'Machine-Learning-in-Atmospheric-Thermodynamics'...
remote: Enumerating objects: 343, done.
remote: Counting objects: 100% (311/311), done.
remote: Compressing objects: 100% (227/227), done.
remote: Total 343 (delta 107), reused 206 (delta 47), pack-reused 32
Receiving objects: 100% (343/343), 180.07 MiB | 17.05 MiB/s, done.
Resolving deltas: 100% (115/115), done.


In [23]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split

path='Machine-Learning-in-Atmospheric-Thermodynamics/hw03'
filen=path+'/data/era5_stn_th.dat'

data=np.fromfile(filen,dtype=np.float32)
data=data.reshape(-1,17)

x=data[:,0:16]
y=data[:,16].reshape(-1,1)

print(x.shape,x[1,:])
print(y.shape,y[1,:])

x_temp,x_valid,y_temp,y_valid=train_test_split(x,y,test_size=0.2,random_state=1)
x_train,x_test,y_train,y_test=train_test_split(x_temp,y_temp,test_size=0.2,random_state=1)

scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)
x_valid=scaler.transform(x_valid)

print('finished')

(9417, 16) [326.37604 323.53775 320.28677 317.47955 315.22504 312.1862  310.70364
 309.3074  307.98483 306.47357 305.08533 303.91504 302.85    301.75815
 300.15994 299.9878 ]
(9417, 1) [0.]
finished


In [24]:
import torch
from torch.utils.data import Dataset, DataLoader

class TrainData(Dataset):
    
    def __init__(self, x_data, y_data):
        self.x_data = x_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.x_data)

train_data=TrainData(torch.FloatTensor(x_train),torch.FloatTensor(y_train))
test_data=TrainData(torch.FloatTensor(x_test),torch.FloatTensor(y_test))

class ValidData(Dataset):
    
    def __init__(self, x_data):
        self.x_data = x_data
        
    def __getitem__(self, index):
        return self.x_data[index]
        
    def __len__ (self):
        return len(self.x_data)
    
valid_data=ValidData(torch.FloatTensor(x_valid))

BATCH_SIZE=64
train_loader=DataLoader(dataset=train_data,batch_size=BATCH_SIZE,shuffle=True)
test_loader=DataLoader(dataset=test_data,batch_size=BATCH_SIZE,shuffle=True)
valid_loader=DataLoader(dataset=valid_data,batch_size=1)

print('finished')

finished


In [39]:
from torch import nn
from torchvision.transforms import ToTensor, Lambda, Compose
import time

class BinaryClassification(nn.Module):
  def __init__(self):
    super(BinaryClassification, self).__init__()
    self.BC = nn.Sequential(
      nn.Linear(16, 128),   
      nn.ReLU(),     
      nn.BatchNorm1d(128),   
      nn.Linear(128, 128),
      nn.ReLU(),
      nn.BatchNorm1d(128),
      nn.Dropout(p=0.1),
      nn.Linear(128, 1)
    )

  def forward(self, x):
    freq = self.BC(x)
    binary = freq #torch.round(freq)
    return binary

print('defined model')

defined model


In [40]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = BinaryClassification().to(device)

loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    model.train()
    train_loss=0

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)  
        loss = loss_fn(pred, y) 
        train_loss += loss.item()

        optimizer.zero_grad()  
        loss.backward()      
        optimizer.step()      
       
        #if batch % 100 == 0:
        #    loss, current = loss.item(), batch * len(X)
        #    print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    train_loss /= num_batches
    return train_loss

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)

    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad(): 
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)

            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (torch.round(torch.sigmoid(pred))==y).sum().float()

    test_loss /= num_batches
    correct /= size
    print(f"Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}")
    return test_loss

# training start
epochs=200

tt=time.time()
train_loss=[]
test_loss=[]
for t in range(epochs):
    print(f"Epoch {t+1}-------------------------------")
    loss = train(train_loader, model, loss_fn, optimizer)
    train_loss.append(loss)
    loss = test(test_loader, model, loss_fn)
    test_loss.append(loss)

elapse=time.time()-tt
print(elapse)

Epoch 1-------------------------------
Accuracy: 51.8%, Avg loss: 0.694560
Epoch 2-------------------------------
Accuracy: 61.2%, Avg loss: 0.644897
Epoch 3-------------------------------
Accuracy: 64.5%, Avg loss: 0.625451
Epoch 4-------------------------------
Accuracy: 67.3%, Avg loss: 0.603757
Epoch 5-------------------------------
Accuracy: 68.1%, Avg loss: 0.605434
Epoch 6-------------------------------
Accuracy: 68.7%, Avg loss: 0.590997
Epoch 7-------------------------------
Accuracy: 70.0%, Avg loss: 0.580488
Epoch 8-------------------------------
Accuracy: 69.7%, Avg loss: 0.579207
Epoch 9-------------------------------
Accuracy: 71.3%, Avg loss: 0.564572
Epoch 10-------------------------------
Accuracy: 71.7%, Avg loss: 0.568700
Epoch 11-------------------------------
Accuracy: 70.7%, Avg loss: 0.575378
Epoch 12-------------------------------
Accuracy: 71.7%, Avg loss: 0.561495
Epoch 13-------------------------------
Accuracy: 72.0%, Avg loss: 0.568593
Epoch 14-------------